## EMI CALCULATOR With Functions

In [1]:
## Importing Required libraries
import pandas as pd
from datetime import datetime

In [2]:
# Function to calculate EMI
def calculate_emi(principal:float , rate:float, time:int) -> float:
    """input: This fxn is taking principal AMOUNT , interest rate and Tenure
        output: monthly EMI 
        Functionailty: This fxn. is calculating emi on basis of given inputs

        Calculate the Equated Monthly Installment (EMI) for a loan.

    Parameters:
        principal (float): Loan amount.
        rate (float): Annual interest rate (in percentage).
        time (int): Loan tenure in months.

    Returns:
        float: The calculated EMI amount.
        """
    rate = rate / (12 * 100)  # monthly interest rate
    emi = (principal * rate * (1 + rate) ** time) / ((1 + rate) ** time - 1)  # EMI formula
    return emi
    

In [5]:
# Function to create date Data frame
def create_df_date_col(tenure_months:int)-> pd.DataFrame:
    """
    Create a DataFrame with date ranges for the given tenure.

    Parameters:
    tenure_months (int): The number of months for the tenure.

    Returns:
    DataFrame: A DataFrame containing dates, months, and years.
    """
    current_year = datetime.today().year # Extracting current Year 
    current_month = datetime.today().month # Extracting current month
    current_date = datetime.today().day # Extracting current date
    dates = pd.date_range(start=f"{current_year}-{current_month}-{current_date}",periods=tenure_months,freq="ME")
   
    df = pd.DataFrame({"Date":dates}) # Creating a dataframe with date range for tenure
    df["Month"] = df["Date"].dt.strftime('%b')  ## Extracting Month
    df["Year"] = df["Date"].dt.strftime('%Y')   ## Extracting Year
    
    return df 


In [7]:
# Loan details
principal = 50000  # Principal loan amount in currency units
annual_rate = 12  # Annual interest rate in percentage
tenure_months =  24  # Loan tenure in months

## Calling fxn to get EMI
emi= calculate_emi(principal,annual_rate,tenure_months)
monthly_rate = annual_rate / (12 * 100)

## creating date tenure 
df = create_df_date_col(tenure_months)

## Create other columns for df
df["Principal_Paid"] = None 
df['Interest_Paid'] = None  # Placeholder for interest paid
df["Total Payment"] = emi
df['Remaining_Balance'] = None  # Placeholder for balance
df["Loan Paid To Date"] = None


In [9]:
# Populate the EMI schedule
remaining_balance = principal
for i in range(len(df)):
    interest_paid  = remaining_balance * monthly_rate
    principal_paid = emi - interest_paid
    remaining_balance-=principal_paid
    
    df.loc[i,'Principal_Paid'] = round(principal_paid,2)
    df.loc[i,"Interest_Paid"]= round(interest_paid,2)
    df.loc[i,"Remaining_Balance"] = round(remaining_balance,2)
    df.loc[i,"Loan Paid To Date"] = f"{round((((principal - remaining_balance) /principal ) * 100 ),2)}%"
    

In [11]:
## Drop Date column as its not required
df.drop(columns=["Date"],inplace=True)

In [13]:
df

,Month,Year,Principal_Paid,Interest_Paid,Total Payment,Remaining_Balance,Loan Paid To Date
0,Feb,2025,1853.67,500.0,2353.673611,48146.33,3.71%
1,Mar,2025,1872.21,481.46,2353.673611,46274.12,7.45%
2,Apr,2025,1890.93,462.74,2353.673611,44383.18,11.23%
3,May,2025,1909.84,443.83,2353.673611,42473.34,15.05%
4,Jun,2025,1928.94,424.73,2353.673611,40544.4,18.91%
5,Jul,2025,1948.23,405.44,2353.673611,38596.17,22.81%
6,Aug,2025,1967.71,385.96,2353.673611,36628.46,26.74%
7,Sep,2025,1987.39,366.28,2353.673611,34641.07,30.72%
8,Oct,2025,2007.26,346.41,2353.673611,32633.81,34.73%
9,Nov,2025,2027.34,326.34,2353.673611,30606.47,38.79%


In [15]:
## Yearly Balance
df.groupby(df["Year"])["Remaining_Balance"].sum().reset_index()

,Year,Remaining_Balance
0,2025,423486.21
1,2026,175330.44
2,2027,0.0


In [17]:
## Showing Aggregate Yearly Matrixes
grouped_schedule = (
    df.groupby('Year', as_index=False)
    .agg({
        'Principal_Paid': 'sum',  # Total Principal Paid
        'Interest_Paid': 'sum',   # Total Interest Paid
        'Total Payment': 'sum',   # Total Payment (Principal + Interest)
        'Remaining_Balance': 'last',  # Remaining Balance at the end of the year
        'Loan Paid To Date':'last'  # Total % of Amount paid
    })
)


In [19]:
grouped_schedule

,Year,Principal_Paid,Interest_Paid,Total Payment,Remaining_Balance,Loan Paid To Date
0,2025,21441.13,4449.25,25890.409723,28558.86,42.88%
1,2026,26228.49,2015.6,28244.083334,2330.37,95.34%
2,2027,2330.37,23.3,2353.673611,0.0,100.0%
